In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# -----------------------------
# 1. Initialize Spark session
# -----------------------------
spark = SparkSession.builder.appName("GoldLayerClaims").getOrCreate()

# -----------------------------
# 2. Load silver_claims_enriched
# -----------------------------
silver_claims = spark.table("meiproject.default.silver_claims_enriched")

# -----------------------------
# 3. Aggregation by Member
# -----------------------------
gold_by_member = silver_claims.groupBy("MemberID", "MemberName").agg(
    F.count("ClaimID").alias("TotalClaims"),
    F.sum("Amount").alias("TotalAmount"),
    F.sum(F.when(F.col("Status") == "Approved", 1).otherwise(0)).alias("ApprovedClaims"),
    F.sum(F.when(F.col("Status") == "Rejected", 1).otherwise(0)).alias("RejectedClaims"),
    F.sum(F.when(F.col("Status") == "Pending", 1).otherwise(0)).alias("PendingClaims"),
    F.collect_list("DiagnosisDescriptions").alias("AllDiagnosisDescriptions")
)

# -----------------------------
# 4. Aggregation by Provider
# -----------------------------
gold_by_provider = silver_claims.groupBy("ProviderID", "ProviderName").agg(
    F.count("ClaimID").alias("TotalClaims"),
    F.sum("Amount").alias("TotalAmount"),
    F.sum(F.when(F.col("Status") == "Approved", 1).otherwise(0)).alias("ApprovedClaims"),
    F.sum(F.when(F.col("Status") == "Rejected", 1).otherwise(0)).alias("RejectedClaims"),
    F.sum(F.when(F.col("Status") == "Pending", 1).otherwise(0)).alias("PendingClaims"),
    F.collect_list("DiagnosisDescriptions").alias("AllDiagnosisDescriptions")
)

# -----------------------------
# 5. Save Gold Tables
# -----------------------------
gold_by_member.write.mode("overwrite").saveAsTable("meiproject.default.gold_claims_by_member")
gold_by_provider.write.mode("overwrite").saveAsTable("meiproject.default.gold_claims_by_provider")
